# DESCARGAR VIDEO YOUTUBE, CON SONIDO Y AÑADIRLE VOZ EN OFF CON TRADUCCIÓN

In [1]:
nombre_video = "Lecture20.mp4"

In [2]:
import os

output_audio = nombre_video.replace(".mp4", ".mp3")
carpeta_transcripcion = "audio_transcription"
fichero_transcripcion = nombre_video.replace(".mp4", ".vtt")
nombre_transcripcion = os.path.join(carpeta_transcripcion, fichero_transcripcion)
nombre_transcripcion_adaptada = os.path.join(carpeta_transcripcion, nombre_video.replace(".mp4", "_adaptados.vtt"))
#se traducirá la transcripción una vez se haya adaptado para mejorar la traducción
nombre_transcripcion_trad = os.path.join(carpeta_transcripcion, nombre_video.replace(".mp4", "_traducidos.vtt"))
#se adaptará la transcripción traducida para que sea más corta para el T2S
nombre_transcripcion_lista_t2s = os.path.join(carpeta_transcripcion, nombre_video.replace(".mp4", "_lista_t2s.vtt"))

video_transcrito_name = nombre_video.replace(".mp4", "_transcribed.mp4")

print(nombre_video)
print(output_audio)
print(nombre_transcripcion)
print(nombre_transcripcion_trad)
print(nombre_transcripcion_adaptada)
print(nombre_transcripcion_lista_t2s)
# print(nombre_transcripcion_es)
print(video_transcrito_name)
!export TF_ENABLE_ONEDNN_OPTS=0 #para desactivar el optimizador que puede dar problemas de accuracy


Lecture20.mp4
Lecture20.mp3
audio_transcription/Lecture20.vtt
audio_transcription/Lecture20_traducidos.vtt
audio_transcription/Lecture20_adaptados.vtt
audio_transcription/Lecture20_lista_t2s.vtt
Lecture20_transcribed.mp4
/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [14]:
from pytube import YouTube

def Download(link):
    youtubeObject = YouTube(link)
    youtubeObject = youtubeObject.streams.get_highest_resolution()
    print("Downloading...")
    try:
        youtubeObject.download(filename=nombre_video)
    except:
        print("An error has occurred")
    print("Download is completed successfully")
    

link = input("Enter the YouTube video URL: ")
Download(link)

Downloading...
Download is completed successfully


In [ ]:
# %%capture
# !apt update
# !pip install git+https://github.com/openai/whisper.git
# !apt install ffmpeg

# Separamos audio de video y pasamos Whisper para sacar la transcripción

In [15]:
import os

#SEPARAMOS EL AUDIO EN OTRO FICHERO (LUEGO LO JUNTAREMOS OTRA VEZ)
!ffmpeg -y -i "$nombre_video" -c:a libmp3lame -qscale:a 1 "$output_audio"

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  

Hacemos la transcripción

In [3]:
import whisper

#generamos la transcripción con whisper
!whisper  {output_audio} --task transcribe --model large --verbose False --output_dir {carpeta_transcripcion}

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Detected language: English
100%|████████████████████████████████| 58326/58326 [01:20<00:00, 720.93frames/s]


Si quisieramos traducir desde otro idioma distinto del inglés como el frances, alemán, chino, etc, podríamos primero usar la traducción automática de la transcripción de Whisper al inglés y luego traducir esta transcripción al español con un modelo traductor del ingles al español

In [16]:
import whisper

#generamos la transcripción con whisper
!whisper  {output_audio} --task translate --model large --verbose False --output_dir {carpeta_transcripcion}

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)


Detected language: German
100%|████████████████████████████████| 49024/49024 [00:53<00:00, 918.95frames/s]


También se puede hacer transcripción al mismo tiempo que traducción desde inglés a español (es más rápida pero la traducción inversa no está demasiado conseguida, para mayor calidad usar transcripción normal y luego convertir al español usando modelo de traducción):

In [1]:
import whisper

#generamos la transcripción con whisper
!whisper  {output_audio} --task transcribe --model large-v2 --language="Spanish" --verbose False --output_dir "./ejemplos"

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Traceback (most recent call last):
  File "/home/javier/miniconda3/envs/bark/lib/python3.10/site-packages/whisper/audio.py", line 59, in load_audio
    out = run(cmd, capture_output=True, check=True).stdout
  File "/home/javier/miniconda3/envs/bark/lib/python3.10/subprocess.py", line 526, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['ffmpeg', '-nostdin', '-threads', '0', '-i', '{output_audio}', '-f', 's16le', '-ac', '1', '-acodec', 'pcm_s16le', '-ar', '16000', '-']' returned non-zero exit status 1.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/javier/miniconda3/envs/bark/bin/whisper", line 8, in <module>
    sys.exit(cli())
  File "/home/javier/miniconda3/envs/bark/lib/python3.10/site-packages/whisper/transcribe.py", line 456, in cli
    re

# PARA TRADUCIR .VTT AL Español podemos usar un modelo de hugginface de tipo Helsinki-NLP:

In [4]:
from transformers import MarianMTModel, MarianTokenizer

def translate_text_to_spanish(text):
    model_name = 'Helsinki-NLP/opus-mt-en-es'  # Modelo para traducir de inglés a español
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    
    tokens = tokenizer(text, return_tensors='pt', padding=True)
    translation = model.generate(**tokens)
    decoded = tokenizer.decode(translation[0], skip_special_tokens=True)
    
    return decoded

text = "This is the story of Diego de la Vega and of how he became the legendary Zorro. At last I am able to reveal his identity, which for so many years we kept secret, though I do so with some unease, since a blank page is more intimidating to me than the naked swords of Moncada’s men."
text = """Science has discovered two, exactly two ways to memorize things.$Mnemonic cues and spaced repetition.$When I say mnemonic cue, I mean an association, like a story.$
"""

# text = """When you do this properly, what happens is that you see the symbol, $ 
# and then that causes you to recall the story that you told yourself or whatever, $
# and the story causes you to recall the other piece of information that you were trying to attach to that symbol. $
# """
translated_text = translate_text_to_spanish(text)
print(f"Traducción: {translated_text}")

2023-09-10 10:04:34.272436: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-10 10:04:34.432692: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Traducción: La ciencia ha descubierto dos, exactamente dos maneras de memorizar las cosas. $ señales mnemónicas y repetición espaciada. $ Cuando digo señal mnemónica, me refiero a una asociación, como una historia. $


In [86]:
# intermedio1 = "".join(translated_text)
# intermedio1.split(" $ ")
# # quitar último $
# intermedio1 = intermedio1.replace(" $", " ")
# print(intermedio1)

La ciencia ha descubierto dos, exactamente dos maneras de memorizar las cosas.  señales mnemónicas y repetición espaciada.  Cuando digo señal mnemónica, me refiero a una asociación, como una historia. 


Para traducir del español al inglés se puede hacer directamente desde Whisper. Este traductor de inglés a español sería útil, sobre todo, para traducción de texto a texto.

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

def translate_text_to_english(text):
    model_name = 'Helsinki-NLP/opus-mt-es-en'  # Modelo para traducir de español a inglés
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    
    tokens = tokenizer(text, return_tensors='pt', padding=True)
    translation = model.generate(**tokens)
    decoded = tokenizer.decode(translation[0], skip_special_tokens=True)
    
    return decoded

text = "Esta es la historia de Diego de la Vega y de cómo se convirtió en el legendario Zorro. Por fin soy capaz de revelar su identidad, que durante tantos años mantuvimos en secreto, aunque lo hago con cierta inquietud, ya que una página en blanco es más intimidante para mí que las espadas desnudas de los hombres de Moncada."
translated_text = translate_text_to_english(text)
print(f"Traducción: {translated_text}")


### Traducción simple: linea por linea:

In [5]:
import re

def traducir(tipo_traductor, text, origen="eng", destino="spa"):
    if tipo_traductor == "helsinki":
        if origen == "eng":
            translated_text = translate_text_to_spanish(text)
        else:
            translated_text = translate_text_to_english(text)
        return translated_text

def traducir_subtitulos(tipo_traductor, input_file, output_file, origen="eng", destino="spa"):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        sentence = ""
        segments_count = 0
        for line in infile:
            if re.match(r'^\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}\.\d{3}', line) or line.strip() == "" or line.strip() == "WEBVTT":
                outfile.write(line)
            else:
                print("line:", line.strip())           
                translated_sentence = traducir(tipo_traductor, line.strip(), origen, destino)
                print("translated_sentence:", translated_sentence, "\n")
                outfile.write(translated_sentence + '\n')


### Mejor traducción: agrupar lineas para tener mejor contexto. (y también traduzca más rápido)

Las lineas se agruparán así:
```bash
"""Science has discovered two, exactly two ways to memorize things. $ 
Mnemonic cues and spaced repetition. $
When I say mnemonic cue, I mean an association, like a story. $


Traducción: Cuando haces esto correctamente, lo que sucede es que ves el símbolo, $ y luego eso te hace recordar la historia que te contaste a ti mismo o lo que sea, $ y la historia te hace recordar la otra pieza de información que estabas tratando de adjuntar a ese símbolo. $
```


Y después se traducirán y volverán a meterse cada una traducida con sus respectivas cabeceras de tiempo.

Aquí el código para hacer esa agrupación mejorando la traducción simple. Lo que hará será meter en cada iteración la lógica que agrupe mínimo N lineas hasta que encutre un ".", las separe por $ y las traduzca, luego separará por $ y las pondrá junto con sus respectivas cabeceras de tiempo.

In [5]:
import re

def traducir(tipo_traductor, text, origen="eng", destino="spa"):
    if tipo_traductor == "helsinki":
        if origen == "eng":
            translated_text = translate_text_to_spanish(text)
        else:
            translated_text = translate_text_to_english(text)
        return translated_text

def traducir_subtitulos(tipo_traductor, input_file, output_file, origen="eng", destino="spa"):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        sentence = ""
        primera_linea = True
        max_paquetes = 6 #número de líneas que se agruparán para traducir
        paquete_count = 0
        buffer_traslator = []
        buffer_cabeceras = []
        agrupando = False
        for line in infile:
            if primera_linea:
                outfile.write(line) #WEBVTT
                primera_linea = False
            es_cabecera_temporal = re.match(r'^\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}\.\d{3}', line)
            if es_cabecera_temporal and paquete_count == 0:
                agrupando = True
                buffer_cabeceras = []
                # outfile.write(line)
            if agrupando:
                if es_cabecera_temporal:
                    buffer_cabeceras.append(line)
                if not es_cabecera_temporal and line.strip() != "":
                    paquete_count += 1
                    buffer_traslator.append(line.strip() + " $ ")
                    if line.strip()[-1] == "." and paquete_count > 3 or paquete_count == max_paquetes:
                        #hemos terminado de agrupar ahora toca traducir y después
                        paquete_count = 0
                        agrupando = False
                        print("\nbuffer_traslator:", "".join(buffer_traslator))
                        translated_sentence = traducir(tipo_traductor, "".join(buffer_traslator), origen, destino)
                        buffer_traslator = []
                        print("translated_sentence:", translated_sentence, "\n")
                        # quitar último $
                        ultimo_dolar_index = translated_sentence.rfind("$")
                        translated_sentence = translated_sentence[:ultimo_dolar_index]

                        # translated_sentence = translated_sentence.replace(" $", " ") 

                        # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
                        translated_sentence_list = translated_sentence.split("$")
                        print("translated_sentence_list:", translated_sentence_list)
                        for i in range(len(translated_sentence_list)):
                            outfile.write(buffer_cabeceras[i])
                            outfile.write(translated_sentence_list[i].strip() + "\n")
                            outfile.write("\n")
                        


        if paquete_count != 0: #hemos terminado pero se han quedado lienas sin escribir
            print("ULTIMO PAQUETE!!!")
            print("\nbuffer_traslator:", buffer_traslator)
            translated_sentence = traducir(tipo_traductor, "".join(buffer_traslator), origen, destino)
            buffer_traslator = []
            print("translated_sentence:", translated_sentence, "\n")
            # quitar último $
            ultimo_dolar_index = translated_sentence.rfind("$")
            translated_sentence = translated_sentence[:ultimo_dolar_index]

            # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
            translated_sentence_list = translated_sentence.split("$")
            print("translated_sentence_list:", translated_sentence_list)
            for i in range(len(translated_sentence_list)):
                outfile.write(buffer_cabeceras[i])
                outfile.write(translated_sentence_list[i].strip() + "\n")     

In [17]:
print("Traducimos de", nombre_transcripcion, "a", nombre_transcripcion_trad)
# podemos elegir entre "seamless" y "helsinki" (helinski es más rápido y más preciso y funciona bien para frases largas)
traducir_subtitulos("helsinki", nombre_transcripcion, nombre_transcripcion_trad, "eng", "spa")

Traducimos de audio_transcription/Lecture20.vtt a audio_transcription/Lecture20_traducidos.vtt

buffer_traslator: He broke into this fixed circle as a rising star of the year, like a whirlwind in which it looked as if there would be no place for a German driver anymore. $ A lot must have happened in half a year. Did you learn a lot in this half a year or was everything actually quite normal? $ I think the prerequisites were certainly given much earlier. $ I started driving a test program with Mercedes-Benz in 1989. $ 


translated_sentence: Él irrumpió en este círculo fijo como una estrella en ascenso del año, como un torbellino en el que parecía como si no habría lugar para un conductor alemán más. $ Mucho debe haber sucedido en medio año. ¿Aprendió mucho en este medio año o era todo en realidad bastante normal? $ Creo que los requisitos previos se dieron sin duda mucho antes. $ Empecé a conducir un programa de pruebas con Mercedes-Benz en 1989. $ 

translated_sentence_list: ['Él irrumpió en este círculo fijo como una estrella en ascenso del año, como un torbellino en el que parecía como si no habría lugar para un conductor alemán más. ', ' Mucho debe haber sucedido en medio año. ¿Aprendió mucho en este medio año o era todo en realidad bastante normal? ', ' Creo que los requisitos previos se dieron sin duda mucho antes. ', ' Empecé a conducir un programa de pruebas con Mercedes-Benz en 1989. ']

buffer_traslator: That was the Mercedes-Benz Junior Team in the Sportwagen World Championship. $ I was abl

Como el generador T2S en español necesita de frases de más de 12 palabras (y menos de 25) para que funciones aceptablemente tenemos que procesar el .VTT para que agrupe

In [19]:
def timestamp_to_seconds(timestamp):
    parts = timestamp.split(':')
    m, s_ms = parts[0], parts[1]
    s, ms = s_ms.split('.') if '.' in s_ms else (s_ms, '000')
    total_seconds = int(m) * 60 + int(s) + int(ms) / 1000
    return total_seconds

def seconds_to_timestamp(seconds):
    m, s = divmod(seconds, 60)
    s_int = int(s)
    ms = int((s - s_int) * 1000)
    return f"{int(m):02d}:{s_int:02d}.{ms:03d}"

# # Probando las funciones
# test_timestamp = "00:05.123"
# test_seconds = 123

# print(f"Conversión de timestamp a segundos: {timestamp_to_seconds(test_timestamp)} segundos")
# print(f"Conversión de segundos a timestamp: {seconds_to_timestamp(test_seconds)}")


def transform_vtt_file(input_file_path, output_file_path):
    try:
        # Leer el contenido del archivo VTT de entrada
        with open(input_file_path, 'r', encoding='utf-8') as file:
            vtt_content = file.read()

        # Aplicar la transformación
        lines = vtt_content.split("\n")
        output_lines = ["WEBVTT"]
        i = 1

        while i < len(lines):
            line = lines[i].strip()
            
            if "-->" in line:  # We have a timestamp line
                start_time = line.split("-->")[0].strip()
                i += 1
                current_text = lines[i].strip()
                i += 1
                
                while i < len(lines) and not "-->" in lines[i]:
                    current_text += " " + lines[i].strip()
                    i += 1
                
                # Check if the current text line has less than 13 words
                if len(current_text.split()) < 13 and i < len(lines) and "-->" in lines[i]:
                    end_time = lines[i].split("-->")[1].strip()
                    i += 1
                    next_text = lines[i].strip()
                    i += 1
                    while i < len(lines) and not "-->" in lines[i]:
                        next_text += " " + lines[i].strip()
                        i += 1
                    current_text += " " + next_text
                else:
                    end_time = line.split("-->")[1].strip()

                output_lines.append(f"{start_time} --> {end_time}")
                output_lines.append(current_text)
                output_lines.append("")  # Adding an empty line to separate subtitles
            else:
                i += 1
                
        transformed_vtt_content = "\n".join(output_lines)

        # Escribir el contenido transformado en el archivo VTT de salida
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.write(transformed_vtt_content)
        
        return "Transformación completada exitosamente."

    except Exception as e:
        return f"Ocurrió un error durante la transformación: {e}"



print("Adaptamos la transcripción para que sea más larga y se pueda generar su voz mejor")
transform_vtt_file(nombre_transcripcion_trad, nombre_transcripcion_adaptada)


Adaptamos la transcripción para que sea más larga y se pueda generar su voz mejor


'Transformación completada exitosamente.'

In [ ]:
# print("Traducimos de", nombre_transcripcion, "a", nombre_transcripcion_trad)
# # podemos elegir entre "seamless" y "helsinki" (helinski es más rápido y más preciso y funciona bien para frases largas)
# traducir_subtitulos("helsinki", nombre_transcripcion_adaptada, nombre_transcripcion_trad, "eng", "spa")

### Otro tipo Traducción más compleja (este tipo desajusta los tiempos)

primero agrupamos los archivos .VTT en lineas más largas que terminan en signo de puntuación para tener más contexto en la traducción

In [ ]:
#NOTA ESTÁ REPETIDA MÁS ARRIBA
def merge_vtt_lines(lines):
    merged_lines = ['WEBVTT\n\n']  # Initialize with the WEBVTT header and an extra new line
    buffer_time_start = None
    buffer_time_end = None
    buffer_text = None

    for line in lines:
        line = line.strip()

        # If the line is a time header
        if re.match(r'\d{2}:\d{2}.\d{3} --> \d{2}:\d{2}.\d{3}', line):
            time_start, time_end = line.split(' --> ')
            
            if buffer_text is not None:
                # Check if the text in the buffer ends with a punctuation mark
                if re.search(r'[.!?;,:]', buffer_text[-1]):
                    # Add the text in the buffer to the list with its corresponding time
                    merged_lines.append(f"{buffer_time_start} --> {buffer_time_end}\n{buffer_text}\n\n")
                    # Reset buffer_time and buffer_text for the next segment
                    buffer_time_start = time_start
                    buffer_time_end = time_end
                    buffer_text = None
                else:
                    # Update the end time of the buffer
                    buffer_time_end = time_end
            
            else:
                buffer_time_start = time_start
                buffer_time_end = time_end
            
            continue  # Skip to the next iteration

        # If the line is a transcription and is not a WEBVTT header
        elif line and not line.lower() == 'webvtt':
            if buffer_text is None:
                buffer_text = line
            else:
                # Add the current line to the text in the buffer
                buffer_text += " " + line

    # Add any remaining text in the buffer
    if buffer_text is not None:
        merged_lines.append(f"{buffer_time_start} --> {buffer_time_end}\n{buffer_text}\n")

    return merged_lines



def adapt_vtt_file(archivo_origen, archivo_adaptado):
    # Leer el archivo origen línea por línea y almacenarlo en una lista
    with open(archivo_origen, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Utilizar la función `merge_vtt_lines` para obtener las líneas adaptadas
    merged_lines = merge_vtt_lines(lines)

    # Escribir las líneas adaptadas en el archivo destino
    with open(archivo_adaptado, 'w', encoding='utf-8') as file:
        file.writelines(merged_lines)




# Uso de ejemplo
adapt_vtt_file(nombre_transcripcion, nombre_transcripcion_adaptada)


Traducimos de inglés a español:

In [ ]:
print("Traducimos de", nombre_transcripcion, "a", nombre_transcripcion_trad)
# podemos elegir entre "seamless" y "helsinki" (helinski es más rápido y más preciso y funciona bien para frases largas)
traducir_subtitulos("helsinki", nombre_transcripcion_adaptada, nombre_transcripcion_trad, "eng", "spa")

# Generación de Voz (traducida) sincronizada con el video a partir de un .vtt

Preparamos para el S2T, los .vtt con cadenas que son demasiado largas para el generador de voz las dividiremos.

In [20]:
import re
import math

def seconds_to_str(seconds):
    # Obtener minutos, segundos y milisegundos
    minutes = int(seconds // 60)
    remaining_seconds = int(seconds % 60)
    milliseconds = int((seconds * 1000) % 1000)
    
    # Convertir a cadena en el formato deseado
    formatted_time = f"{minutes:02d}:{remaining_seconds:02d}.{milliseconds:03d}"
    return formatted_time

def split_long_lines(input_filename, output_filename):
    # Leer el archivo VTT de entrada
    with open(input_filename, 'r', encoding='utf-8') as f:
        vtt_text = f.read()

    # Regex para encontrar y separar los componentes de cada entrada VTT
    pattern = re.compile(r'(\d{2}:\d{2}.\d{3} --> \d{2}:\d{2}.\d{3})\n(.+?)\n', re.DOTALL)
    entries = pattern.findall(vtt_text)

    # Preparar la lista para las nuevas entradas
    new_entries = []

    # Recorrer cada entrada
    for entry in entries:
        start_time, end_time = entry[0].split(' --> ')
        start_time = int(start_time.split(':')[0]) * 60 + int(start_time.split('.')[0].split(':')[1])  + float(start_time.split('.')[1])/1000
        end_time = int(end_time.split(':')[0]) * 60 + int(end_time.split('.')[0].split(':')[1])  + float(end_time.split('.')[1])/1000

        time_difference = end_time - start_time

        text = entry[1]
        words = text.split()

        if len(words) >= 25:
            # Buscar el signo de puntuación que divide la línea más equitativamente
            best_index = -1
            best_diff = float('inf')

            for i, word in enumerate(words):
                if word[-1] in ('.', ';', ',', ':'):
                    diff = abs(len(words[:i+1]) - len(words[i+1:]))
                    if diff < best_diff:
                        best_diff = diff
                        best_index = i

            first_half_words = words[:best_index+1]
            second_half_words = words[best_index+1:]

            # Calcular los nuevos tiempos
            total_words = len(first_half_words) + len(second_half_words)
            first_end_time = start_time + time_difference * (len(first_half_words) / total_words)
            first_end_time = round(first_end_time, 3)

            # Agregar las nuevas entradas
            new_entries.append((f"{seconds_to_str(start_time)} --> {seconds_to_str(first_end_time)}", ' '.join(first_half_words)))
            new_entries.append((f"{seconds_to_str(first_end_time)} --> {seconds_to_str(end_time)}", ' '.join(second_half_words)))
        else:
            new_entries.append(entry)

    # Escribir las nuevas entradas al archivo VTT de salida
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write("WEBVTT\n\n")
        for entry in new_entries:
            f.write(f"{entry[0]}\n{entry[1]}\n\n")


# simplifico lineas largas
split_long_lines(nombre_transcripcion_adaptada, nombre_transcripcion_lista_t2s)
# lo hacemos por 2 veces para que no quede ninguna linea con mas de 25 palabras
# split_long_lines(nombre_transcripcion_lista_t2s, nombre_transcripcion_lista_t2s)


In [ ]:
split_long_lines(nombre_transcripcion_lista_t2s, nombre_transcripcion_lista_t2s)

## Preprocesamiento antes del T2S para ayudar a las limitaciones del modelo T2S

In [21]:
from pydub import AudioSegment, silence

def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
    """
    Elimina silencios largos de un archivo de audio.

    Parámetros:
    - input_filepath: ruta al archivo de audio de entrada (MP3).
    - output_filepath: ruta al archivo de audio de salida (MP3).
    - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
    - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
    - silence_thresh: umbral de silencio (en dB).
    """
    
    # Cargar el archivo de audio
    audio_segment = AudioSegment.from_wav(input_filepath)
    
    # Encuentra los segmentos de audio separados por silencios
    segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

    # Crear un nuevo segmento de audio con silencios ajustados
    new_audio_segment = AudioSegment.empty()
    silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

    # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
    for segment in segments:
        new_audio_segment += segment + silence_chunk

    # Removemos el último chunk de silencio añadido
    new_audio_segment = new_audio_segment[:-new_silence_len]

    # Guarda el nuevo archivo de audio
    new_audio_segment.export(output_filepath, format="wav")


Para evaluar las generaciones de voz podemos usar whisper para volver a hacer la transcripción de la voz generada y ver si coincide con la original

In [146]:
import whisper
modelWhisper = whisper.load_model('medium')

In [138]:
import Levenshtein

texto1= "hola como estas"
texto2= "hola, como estas?"
# funcion que quita simbolos de puntuacion
def quitar_puntuacion(texto):
    texto = texto.replace('.', '')
    texto = texto.replace(',', '')
    texto = texto.replace('?', '')
    texto = texto.replace('!', '')
    texto = texto.replace('¿', '')
    texto = texto.replace('¡', '')
    texto = texto.replace(';', '')
    texto = texto.replace(':', '')
    texto = texto.replace('(', '')
    texto = texto.replace(')', '')
    texto = texto.replace('[', '')
    texto = texto.replace(']', '')
    texto = texto.replace('{', '')
    texto = texto.replace('}', '')
    texto = texto.replace('"', '')
    texto = texto.replace("'", '')
    texto = texto.replace('`', '')
    texto = texto.replace('´', '')
    texto = texto.replace('’', '')
    texto = texto.replace('‘', '')
    texto = texto.replace('“', '')
    texto = texto.replace('”', '')
    texto = texto.replace('…', '')
    texto = texto.replace('«', '')
    texto = texto.replace('»', '')
    texto = texto.replace('–', '')
    texto = texto.replace('—', '')
    texto = texto.replace('−', '')
    texto = texto.replace('‐', '')
    texto = texto.replace('⁃', '')
    texto = texto.replace('‒', '')
    texto = texto.replace('―', '')
    texto = texto.replace('‹', '')
    texto = texto.replace('›', '')
    texto = texto.replace('•', '')
    texto = texto.replace('·', '')
    texto = texto.replace('‣', '')
    texto = texto.replace('⁌', '')
    texto = texto
    return texto

# funcion que calcula la distancia entre dos textos
Levenshtein.ratio(quitar_puntuacion(texto1), quitar_puntuacion(texto2))

1.0

El modelo no entiende de números aritméticos

In [22]:
import re
from num2words import num2words

def number_to_words(num_str):
    try:
        num = int(num_str)
        return num2words(num, lang='es')
    except ValueError:
        return "Por favor, introduzca un número válido."

def process_numbers_in_line(line):
    def replace_with_words(match):
        return number_to_words(match.group())
    
    return re.sub(r'\b\d+\b', replace_with_words, line)

# Ejemplo de uso
line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
new_line = process_numbers_in_line(line)
print(new_line)  
# Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."

Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas.


In [23]:
# Diccionario con las traducciones


def process_abrev(line):
    translations = {
    'Dr': 'doctor',
    'Sr': 'señor',
    'Sra': 'señora',
    # Añade más traducciones aquí
}
    for abbr, full in translations.items():
        line = line.replace(f'{abbr}.', full)
        line = line.replace(f'{abbr} ', f'{full} ')
    return line

# Ejemplo de uso
line1 = 'El Dr. está hablando con el Sr. Pérez y la Sra. Gómez.'
line2 = 'Buenos días Sr. Gómez.'

new_line1 = process_abrev(line1)
new_line2 = process_abrev(line2)

print(new_line1)  # Debería imprimir: 'El Doctor está hablando con el Señor Pérez y la Señora Gómez.'
print(new_line2)  # Debería imprimir: 'Buenos días Señor Gómez.'

El doctor está hablando con el señor Pérez y la señora Gómez.
Buenos días señor Gómez.


In [24]:
# Diccionario con las traducciones


def otras_traducciones(line):

    translations = {
    '-': ',',
    '—': ',' 
    # Añade más traducciones aquí
    }
    
    for old, new in translations.items():
        line = line.replace(old, new)
    return line

# Ejemplo de uso
line1 = 'Hola —cómo estás—?'
line2 = "El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón —el primer hobbit que fue famoso en el mundo entero— y que él tituló Historia de una ida y de una vuelta,"

new_line1 = otras_traducciones(line1)
new_line2 = otras_traducciones(line2)

print(new_line1)  # Debería imprimir: 'Hola, cómo estás?'
print(new_line2)  # Debería imprimir: 'Bien, gracias.'

Hola ,cómo estás,?
El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón ,el primer hobbit que fue famoso en el mundo entero, y que él tituló Historia de una ida y de una vuelta,


In [25]:
def preprocesado_al_modelo(line):
    line_with_numbers = process_numbers_in_line(line)
    line_with_both = process_abrev(line_with_numbers)
    line_with_all = otras_traducciones(line_with_both)
    return line_with_all

## modelo generador de voz

In [143]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
print(os.environ['TF_ENABLE_ONEDNN_OPTS'])  # Salida: mi_valor

0


Cargamos el modelo generador fairseq

In [26]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd
import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj
    

# Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-es-css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]

# Movemos el modelo al dispositivo GPU
model = model.to('cuda:0')

# Actualizamos la configuración con los datos del task
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

# Creamos el generador
generator = task.build_generator([model], cfg)


# Texto a convertir en voz
text = "Hola, esta es una prueba. Quiero saber hasta dónde llega el modelo."

# Preparamos los datos de entrada para el modelo
sample = TTSHubInterface.get_model_input(task, text)

# Movemos los datos al dispositivo GPU
sample = move_to_device(sample, 'cuda:0')

# Realizamos la predicción
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

# Convertimos el resultado al formato de audio y lo reproducimos
ipd.Audio(wav.cpu(), rate=rate)  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio


2023-09-10 10:20:14 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

2023-09-10 10:20:18 | INFO | fairseq.tasks.speech_to_text | dictionary size (spm_char.txt): 107
2023-09-10 10:20:18 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin
2023-09-10 10:20:20 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin


Definimos interfaz que vamos a usar para usar el modelo y generar voz

In [27]:
def spanish_t2s(text):
    text = preprocesado_al_modelo(text)
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, text)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

    return wav, rate

In [ ]:
import torchaudio
import IPython.display as ipd


texto_prueba = preprocesado_al_modelo("Hola Sr. usuario, esta es una prueba de 2 pasos. Quiero saber hasta dónde llega el modelo.")
print("texto_prueba:", texto_prueba)

wav, rate = spanish_t2s(texto_prueba)

if len(wav.shape) == 1:
    wav = wav.unsqueeze(0)

temp_file_name = "ejemploTemporal.wav"
torchaudio.save(temp_file_name, wav.to('cpu'), rate)


prediction = modelWhisper.transcribe("ejemploTemporal.wav", language="Spanish", temperature=0, beam_size=1)["text"]
print("prediction:", prediction)
similitud = Levenshtein.ratio(quitar_puntuacion(texto_prueba), quitar_puntuacion(prediction))

print("similitud:", similitud)
# escuchar audio
ipd.Audio(wav.cpu(), rate=rate)  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio

De un fichero .vtt crea una lista de pares (tiempo, texto)

In [29]:
import re

def vtt_to_couple_time_text_list(filename):
    # Leemos el contenido del archivo
    with open(filename, 'r', encoding='utf-8') as f:
        vtt_content = f.read()

    # Usamos una expresión regular para identificar y eliminar las líneas de tiempo
    pattern = re.compile(r'^\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}\.\d{3}$')
    
    # Dividimos el contenido por líneas
    lines = vtt_content.split('\n')

    # Filtramos para quedarnos solo con las líneas de tiempo
    time_lines = [line for line in lines if pattern.match(line)]

    # Filtramos las líneas que no queremos: líneas de tiempo y líneas vacías
    text_lines = [line for line in lines if not pattern.match(line) and line.strip() != "" and line != "WEBVTT"]

    # creamos una nueva lista de pares (tiempo, texto)
    time_text_pairs = list(zip(time_lines, text_lines))

    return time_text_pairs

In [ ]:
# import os
# import torchaudio
# import Levenshtein
# import shutil


# small_temp_files = []

# # crear carpeta temporal si no existe
# carpetaTemporal = "./asudioLibro_sp"
# if not os.path.exists(carpetaTemporal):
#     os.makedirs(carpetaTemporal)

# # Itera sobre cada párrafo
# for i, text in enumerate(parrafos):
#     print(f"Paragraph {i + 1}:\n{text}\n")


#     text = parrafos[i]
#     text = preprocesado_al_modelo(text)

#     # text = "pues, contaba el viaje de Bilbo hacia el este y la vuelta,"
#     print(f"Paragraph {i + 1}:\n{text}\n")
#     # Genera el audio

#     wav, rate = spanish_t2s(text)

#     if len(wav.shape) == 1:
#         wav = wav.unsqueeze(0)

#     temp_file_name = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.wav"
#     torchaudio.save(temp_file_name, wav.to('cpu'), rate)

#     similitud = 0
#     contador = 0
#     line = text
#     longitud_texto = len(text.split())
#     while True:
        
#         prediction = modelWhisper.transcribe(temp_file_name, language="Spanish", temperature=0, beam_size=1)["text"]
#         similitud = Levenshtein.ratio(quitar_puntuacion(line), quitar_puntuacion(prediction))
#         print("Linea: ", line)
#         print("Prediction: ", prediction)
#         print("Similitud: ", similitud)
#         print("Contador: ", contador)
        

#         print("Provisional transcripted_sentence:", prediction)
#         if (similitud > 0.96) or (similitud > 0.93 and longitud_texto < 24) or (similitud > 0.90 and contador > 8) or (similitud > 0.88 and contador > 11) or (similitud > 0.85 and contador > 15) or contador > 17:
#             break
#         else:
#             contador += 1
#             if contador > 2 and longitud_texto < 24 or contador > 4:
#                 line = " ".join(text.split()[:contador]) + "," + " ".join(text.split()[contador:])
#             wav, rate = spanish_t2s(line)
#             if len(wav.shape) == 1:
#                 wav = wav.unsqueeze(0)
#             torchaudio.save(temp_file_name, wav.to('cpu'), rate)

#     temp_file_name_definitivo = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.mp3"
#     # convertir .wav en .mp3
#     !ffmpeg -y -i "{temp_file_name}" "{temp_file_name_definitivo}"
#     # eliminar .wav
#     !rm "{temp_file_name}"
#     # añadir fichero a la lista de ficheros temporales
#     small_temp_files.append(temp_file_name_definitivo)


# # Concatena todos los archivos mp3 grandes en uno final
# !ffmpeg -y -f concat -safe 0 -i <(for f in {' '.join(small_temp_files)}; do echo file ${"PWD"}/${"f"}; done) -c copy "{carpetaTemporal}/audioLibro_output.mp3"

# # quitamos silencios
# quitar_silencios(f"{carpetaTemporal}/audioLibro_output.mp3", f"{carpetaTemporal}/audioLibro_output_sin_silencios.mp3")
# !cp "{carpetaTemporal}/audioLibro_output_sin_silencios.mp3" "./audioLibro_sp_output_sin_silencios.mp3"
# shutil.rmtree(carpetaTemporal)

A partir de un fichero .vtt ya traducido (en este caso en inglés) te crea el audio sincronizado

In [30]:
from pydub import AudioSegment
import torchaudio
import os
from datetime import datetime, timedelta


def time_to_milliseconds(time_str):
    time_obj = datetime.strptime(time_str, '%M:%S.%f')
    # time_obj = datetime.strptime(time_str, '%H:%M:%S.%f')
    # return int(timedelta(hours=time_obj.hour, minutes=time_obj.minute, seconds=time_obj.second, microseconds=time_obj.microsecond).total_seconds() * 1000)
    return int(timedelta(minutes=time_obj.minute, seconds=time_obj.second).total_seconds() * 1000)

carpetaTemporal = "./aProfesionalTraductor_sp"

# crear carpeta temporal si no existe
if not os.path.exists(carpetaTemporal):
    os.makedirs(carpetaTemporal)

small_temp_files = []

current_time_ms = 0

paragraphs = vtt_to_couple_time_text_list(nombre_transcripcion_lista_t2s)

delta = 0
silencio = 0
acumulado = 0
umbral = 100 #milisegundos
playback_speed_limit = 1.1
silence_index = 0.5

# Itera sobre cada par (tiempo, texto) en la lista
for i, (time_range, text) in enumerate(paragraphs):
    start_time, end_time = time_range.split(' --> ')
    start_time_ms = time_to_milliseconds(start_time)
    end_time_ms = time_to_milliseconds(end_time)

    # Genera el audio
    # !python3 tortoise/do_tts.py --voice=$"{voice}" --candidates=1 --text=$"{text}" --preset=$"{preset}" --output_path=./$"{voice}"/
    wav, rate = spanish_t2s(text)

    if len(wav.shape) == 1:
        wav = wav.unsqueeze(0)

    temp_file_name = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.wav"
    torchaudio.save(temp_file_name, wav.to('cpu'), rate)

    # quitamos silencios
    quitar_silencios(temp_file_name, temp_file_name, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)

    # obtiene audio_segment a partir de un archivo wav
    audio_segment = AudioSegment.from_wav(temp_file_name)

    # frecuencia de muestreo
    print("sample_rate:", audio_segment.frame_rate)

    # eliminar archivo mp3
    # !rm ./{voice}/{voice}_0.wav

    # gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)
    # audio_segment = gen.squeeze(0).cpu()

    # En pydub la función len() devuelve la duración en milisegundos
    audio_duration_ms = len(audio_segment)

    print("segmento:", i)
    # Calcula la diferencia de tiempo
    # delta = end_time_ms - (start_time_ms + audio_duration_ms)
    # print("delta:", delta)
    # delta += acumulado
    # print("delta acumulado:", delta)

    print("start_time_ms", start_time_ms)
    print("end_time_ms", end_time_ms)
    video_duration_ms = end_time_ms - start_time_ms
    print("video_duration_ms", video_duration_ms)
    print("audio_duration_ms", audio_duration_ms)

    delta = video_duration_ms - audio_duration_ms
    print("delta:", delta)
    acumulado += delta
    print("acumulado:", acumulado)

    # En el caso especial de la primera iteración hay que añadir un silencio al principio
    if i == 0 and delta > 0:
        silence_segment = AudioSegment.silent(duration = delta * 0.9)
        audio_segment = silence_segment + audio_segment
        # desacumulamos el tiempo silenciado
        acumulado = delta * (1 - 0.9)
        print("silencio: ", delta * 0.9)
    else:

        # Ajusta la velocidad si es necesario (dejamos un umbral para agilizar el proceso)
        if acumulado < -umbral:
            multiplicador = 1 +  (abs(acumulado) / audio_duration_ms)
            if multiplicador > playback_speed_limit:
                playback_speed = playback_speed_limit
                # acumulado = -(multiplicador - playback_speed_limit)*audio_duration_ms
            else:
                playback_speed = multiplicador
            #acumulamos el tiempo que nos falta para llegar al tiempo del video
            acumulado += (multiplicador - playback_speed) * audio_duration_ms
            audio_segment = audio_segment.speedup(playback_speed = playback_speed)
            print("acelerado: x", playback_speed)
        elif acumulado > umbral:
            if acumulado < 1000:
                silence_index = 0.5
            elif acumulado < 2000:
                silence_index = 0.65
            elif acumulado < 3000:
                silence_index = 0.75
            else:
                silence_index = 0.90
            silencio = acumulado * silence_index
            # silence_segment = AudioSegment.silent(duration = silencio)
            # si el texto termina en "." añadimos todo el silencio extra al final del segmento
            if text[-1] == ".":
            # añadimos solo la mitad de delta para intentar luego compensar cuando se acelera
                audio_segment =  audio_segment + AudioSegment.silent(duration = silencio) 
            else:
                #lo añadimos la mitad al principio y la otra mitad al final
                audio_segment = AudioSegment.silent(duration = silencio / 2) + audio_segment + AudioSegment.silent(duration = silencio / 2)
            # desacumulamos el tiempo silenciado
            acumulado -= silencio
            # audio_segment += silence_segment
            print("silencio: ", silencio)
    
    print("acumuladoFinal:", acumulado)
        
 
    small_temp_file_name = f'./{carpetaTemporal}/small_temp_audio_{str(i).zfill(5)}.mp3'
    # Exportar como MP3
    audio_segment.export(small_temp_file_name, format="mp3")
    # torchaudio.save(temp_file_name, audio_segment, 24000, format="mp3")
    small_temp_files.append(small_temp_file_name)

    # Eliminar archivo .wav
    os.remove(temp_file_name)


# Concatena todos los archivos mp3 grandes en uno final
!ffmpeg -y -f concat -safe 0 -i <(for f in {' '.join(small_temp_files)}; do echo file ${"PWD"}/${"f"}; done) -c copy "./{carpetaTemporal}/ProfesionalTraductor_output.mp3"

!cp "./{carpetaTemporal}/ProfesionalTraductor_output.mp3" ../



# Elimina archivos mp3 grandes
os.system(f"rm {' '.join(small_temp_files)}")


sample_rate: 22050
segmento: 0
start_time_ms 0
end_time_ms 5000
video_duration_ms 5000
audio_duration_ms 4690
delta: 310
acumulado: 310
silencio:  279.0
acumuladoFinal: 30.999999999999993
sample_rate: 22050
segmento: 1
start_time_ms 5000
end_time_ms 13000
video_duration_ms 8000
audio_duration_ms 4610
delta: 3390
acumulado: 3421.0
silencio:  3078.9
acumuladoFinal: 342.0999999999999
sample_rate: 22050
segmento: 2
start_time_ms 13000
end_time_ms 22000
video_duration_ms 9000
audio_duration_ms 7105
delta: 1895
acumulado: 2237.1
silencio:  1677.8249999999998
acumuladoFinal: 559.2750000000001
sample_rate: 22050
segmento: 3
start_time_ms 22000
end_time_ms 33000
video_duration_ms 11000
audio_duration_ms 9056
delta: 1944
acumulado: 2503.275
silencio:  1877.4562500000002
acumuladoFinal: 625.8187499999999
sample_rate: 22050
segmento: 4
start_time_ms 33000
end_time_ms 40000
video_duration_ms 7000
audio_duration_ms 4623
delta: 2377
acumulado: 3002.81875
silencio:  2702.536875
acumuladoFinal: 300.281

0

Baja el volumen del audio original

In [31]:
from pydub import AudioSegment

# Cargar el segmento de audio desde un archivo mp3
audio_segment = AudioSegment.from_mp3(output_audio)

# Bajar el volumen en 10 dB
audio_segment_quiet = audio_segment - 23

# Guardar el nuevo segmento de audio a un archivo
audio_segment_quiet.export("volumen_disminuido.mp3", format="mp3")

<_io.BufferedRandom name='volumen_disminuido.mp3'>

Creamos video con voz en off traduciendo

In [32]:
!ffmpeg -y -i "{nombre_video}" -i "volumen_disminuido.mp3" -i "{carpetaTemporal}/ProfesionalTraductor_output.mp3" \
-filter_complex "[1:a][2:a]amix=inputs=2:duration=first[aout]" \
-map 0:v -map "[aout]" -c:v copy -c:a aac {nombre_video}_con_audio_mezclado.mp4

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  

frame=14707 fps=2897 q=-1.0 Lsize=   23077kB time=00:08:10.24 bitrate= 385.6kbits/s speed=96.6x    
video:14816kB audio:7817kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.962644%
[aac @ 0x55ab82a8cc00] Qavg: 12086.909


# PRUEBAS VARIAS

Cambiar el pitch:

In [39]:
import librosa
from scipy.io.wavfile import write

# Cargar el archivo de audio
y, sr = librosa.load('tortoise-tts/tortoise/voices/javier/audiotortoise2.wav', sr=None)

# Cambiar el tono del audio sin alterar la duración
y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=1)  # Aumentar el tono #n_steps=-1 mas grave #n_steps=1 mas agudo

# Guardar el audio modificado
write('shifted_output.wav', sr, y_shifted.astype('float32'))


Cambiar escpectograma

In [69]:
import pyworld as pw
import numpy as np
import librosa
import soundfile as sf

# Cargar el archivo de audio
file_path = "tortoise-tts/tortoise/voices/javier/audiotortoise2.wav"
x, fs = librosa.load(file_path, sr=22050)

# Descomposición de WORLD
f0, sp, ap = pw.wav2world(x.astype('double'), fs)

# Aumentar la frecuencia fundamental para feminizar la voz
f0[f0>0] *= 1.8

# Usar el manipulador de formantes de WORLD para modificar los formantes
# sp = pw.cheaptrick(x.astype('double'), f0, np.linspace(0, 0.5, len(x), endpoint=False), fs)

# Reconstruir la señal de audio con los componentes modificados
y = pw.synthesize(f0, sp, ap, fs)

# Guardar la señal de audio modificada
sf.write('voice_feminized.wav', y, fs)


Reverberación

In [47]:
import librosa
from scipy.signal import convolve
import numpy as np
import soundfile as sf

# Carga tu archivo de audio
y, sr = librosa.load('tortoise-tts/tortoise/voices/javier/audiotortoise2.wav', sr=None)

# Crea una respuesta al impulso simple para la reverberación (esto es bastante básico y podrías querer usar una respuesta al impulso más realista)
def simple_reverb_ir(duration, sr, decay=0.005):
    t = np.linspace(0, duration, int(sr * duration))
    ir = np.exp(-decay * t)
    return ir

# Crea una respuesta al impulso y convolútala con tu señal de audio
ir = simple_reverb_ir(0.1, sr)
y_reverb = convolve(y, ir)

# Normalizar el audio para evitar clipping
y_reverb = y_reverb / np.max(np.abs(y_reverb), axis=0)

# Guarda el audio resultante
sf.write('output_with_reverb.wav', y_reverb, sr)


In [ ]:
# antiguo codigo para juntar lineas del .vtt para traudcir
import re

def traducir(tipo_traductor, text, origen="eng", destino="spa"):
    if tipo_traductor == "helsinki":
        if origen == "eng":
            translated_text = translate_text_to_spanish(text)
        else:
            translated_text = translate_text_to_english(text)
        return translated_text

def traducir_subtitulos(tipo_traductor, input_file, output_file, origen="eng", destino="spa"):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        sentence = ""
        primera_linea = True
        max_paquetes = 4 #número de líneas que se agruparán para traducir
        paquete_count = 0
        buffer_traslator = []
        buffer_cabeceras = []
        agrupando = False
        for line in infile:
            if primera_linea:
                outfile.write(line) #WEBVTT
                primera_linea = False
            es_cabecera_temporal = re.match(r'^\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}\.\d{3}', line)
            if es_cabecera_temporal and paquete_count == 0:
                agrupando = True
                buffer_cabeceras = []
                # outfile.write(line)
            if agrupando:
                if es_cabecera_temporal:
                    buffer_cabeceras.append(line)
                if not es_cabecera_temporal and line.strip() != "":
                    paquete_count += 1
                    buffer_traslator.append(line.strip() + " $")
                    if paquete_count == max_paquetes:
                        #hemos terminado de agrupar ahora toca traducir y después
                        paquete_count = 0
                        agrupando = False
                        print("\nbuffer_traslator:", "".join(buffer_traslator))
                        translated_sentence = traducir(tipo_traductor, "".join(buffer_traslator), origen, destino)
                        buffer_traslator = []
                        print("translated_sentence:", translated_sentence, "\n")
                        # quitar último $
                        ultimo_dolar_index = translated_sentence.rfind("$")
                        translated_sentence = translated_sentence[:ultimo_dolar_index]

                        # translated_sentence = translated_sentence.replace(" $", " ") 

                        # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
                        translated_sentence_list = translated_sentence.split("$")
                        print("translated_sentence_list:", translated_sentence_list)
                        for i in range(len(translated_sentence_list)):
                            outfile.write(buffer_cabeceras[i])
                            outfile.write(translated_sentence_list[i].strip() + "\n")
                            outfile.write("\n")
                        


        if paquete_count != 0: #hemos terminado pero se han quedado lienas sin escribir
            print("ULTIMO PAQUETE!!!")
            print("\nbuffer_traslator:", buffer_traslator)
            translated_sentence = traducir(tipo_traductor, "".join(buffer_traslator), origen, destino)
            buffer_traslator = []
            print("translated_sentence:", translated_sentence, "\n")
            # quitar último $
            ultimo_dolar_index = translated_sentence.rfind("$")
            translated_sentence = translated_sentence[:ultimo_dolar_index]

            # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
            translated_sentence_list = translated_sentence.split("$")
            print("translated_sentence_list:", translated_sentence_list)
            for i in range(len(translated_sentence_list)):
                outfile.write(buffer_cabeceras[i])
                outfile.write(translated_sentence_list[i].strip() + "\n")   